# Synthesizing data with time-sensitive location data

## Reconstructing books/members/events

In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *

In [4]:
def get_book_id(x): return x.split('/books/',1)[1][:-1] if '/books/' in x else ''

@cache
def get_books_df(): 
    df=get_urlpath_df('books').fillna('')
    df['book_id']=df.uri.apply(get_book_id)
    return df.set_index('book_id')

In [5]:
df_books=get_books_df()
# df_books

In [6]:
def get_member_id(x): return x.split('/members/',1)[1][:-1] if '/members/' in x else ''

@cache
def get_members_df(): 
    df=get_urlpath_df('members').fillna('')
    df['is_expat'] = df['nationalities'].apply(lambda x: 'France' not in x)
    df['has_wikipedia'] = df['wikipedia_url']!=''
    df['has_viaf'] = df['viaf_url']!=''
    df['birth_decade'] = [str(x)[:3]+'0s' if x else '' for x in df['birth_year']]
    df['generation'] = df['birth_year'].apply(parse_generation)
    df['member_id']=df.uri.apply(get_member_id)
    return df.set_index('member_id')

In [7]:
df_members = get_members_df()
# df_members

In [8]:
def get_event_id(row):
    base=f'{row.member_id} {(row.event_type+"s").upper()} {row.book_id}'
    if not row.start_date: return base
    if row.end_date and row.start_date!=row.end_date:
        return f'{base} FROM {row.start_date} TO {row.end_date}'
    return f'{base} ON {row.start_date}'

@cache
def get_events_df(): 
    # get as exists
    df = get_urlpath_df('events').fillna('')
    
    ## expand by semi colon!
    def getrowinfo(row, i):
        odx={}
        for k,v in dict(row).items():
            if ';' in str(v):
                vs = str(v).split(';')
                v=vs[i]
            odx[k]=v
        return odx
    
    df_events_expanded = pd.DataFrame(
        {**getrowinfo(row,mi), 'member_uri':muri, 'member_id':get_member_id(muri), 'book_id':get_book_id(row.item_uri)}
        for i,row in df.iterrows()
        for mi,muri in enumerate(row.member_uris.split(';'))
    )
    
    odf = df_events_expanded[['member_id','book_id']+[col for col in df if not col.split('_')[0] in {'member','item'}]]
    odf['event_id'] = odf.apply(get_event_id,axis=1)
    
    odf['dwelling_ids'] = odf.apply(lambda row: find_dwelling_id(row, verbose=False), axis=1)
    return odf.set_index('event_id')


In [9]:
df_events = get_events_df()
df_events

NameError: name 'find_dwelling_id' is not defined

## Finding locations

In [8]:
def get_dwelling_id(row):
    o=f'{row.member_id} DWELT AT {row.street_address.replace(" ","-")}'
    if row.start_date: o+=f' FROM {row.start_date}'
    if row.end_date: o+=f' UNTIL {row.end_date}'
    return o
    
@cache
def get_dwellings_df(): 
    df=get_urlpath_df('dwellings').fillna('')
    startcols = ['member_id','start_date','end_date']
    endcols = [c for c in df if c.endswith('_id')]
    df['member_id']=df.member_uri.apply(get_member_id)
    odf=df[startcols + [c for c in df if c not in set(startcols+endcols)] + endcols]
    odf['dwelling_id'] = odf.apply(get_dwelling_id, axis=1)
    odf['lat']=odf.latitude.apply(lambda x: float(x) if x else np.nan)
    odf['lon']=odf.longitude.apply(lambda x: float(x) if x else np.nan)
    odf['arrond_id']=odf.arrrondissement.apply(lambda x: str(int(x)) if x else '')
    return odf.set_index('dwelling_id')
    

In [47]:
df_dwellings = get_dwellings_df()
df_dwellings

,member_id,start_date,end_date,member_uri,start_date_precision,end_date_precision,street_address,city,postal_code,latitude,...,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id,lat,lon,arrond_id
dwelling_id,,,,,,,,,,,,,,,,,,,,,
abbott-berenice DWELT AT 44-rue-du-Bac,abbott-berenice,,,https://shakespeareandco.princeton.edu/members...,,,44 rue du Bac,Paris,75007,48.85681,...,7.0,9985,7151,1237,121,,2.0,48.85681,2.32644,7
abbott DWELT AT 246-boulevard-Raspail,abbott,,,https://shakespeareandco.princeton.edu/members...,,,246 boulevard Raspail,Paris,75014,48.83818,...,14.0,2244,1583,1238,444,,2.0,48.83818,2.33049,14
ernst-abel DWELT AT 36-rue-Bonaparte,ernst-abel,,,https://shakespeareandco.princeton.edu/members...,,,36 rue Bonaparte,Paris,75006,48.85509,...,6.0,9996,7158,1239,192,,2.0,48.85509,2.33358,6
yvonne-abelin DWELT AT 7-rue-de-l’Odéon,yvonne-abelin,,,https://shakespeareandco.princeton.edu/members...,,,7 rue de l’Odéon,Paris,75006,48.85118,...,6.0,10000,7161,1240,917,,2.0,48.85118,2.33879,6
ably-2 DWELT AT 10-bis-avenue-de-la-Grande-Armée,ably-2,,,https://shakespeareandco.princeton.edu/members...,,,10 bis avenue de la Grande Armée,Paris,75017,48.81119,...,17.0,1288,902,1241,834,,2.0,48.81119,2.27050,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
van-den-bergh DWELT AT 9-avenue-Frédéric-le-Play,van-den-bergh,,,https://shakespeareandco.princeton.edu/members...,,,9 avenue Frédéric le Play,Paris,75007,48.85433,...,7.0,562,5096,789,731,,2.0,48.85433,2.30431,7
van-den-bergh DWELT AT,van-den-bergh,,,https://shakespeareandco.princeton.edu/members...,,,,Courmayeur,,45.79692,...,,562,5096,790,732,,5.0,45.79692,6.96896,
van-den-bergh DWELT AT 72-avenue-de-La-Bourdonnais,van-den-bergh,,,https://shakespeareandco.princeton.edu/members...,,,72 avenue de La Bourdonnais,Paris,75007,48.85503,...,7.0,562,5096,791,733,,2.0,48.85503,2.30394,7


In [49]:
df_dwellings[df_dwellings.member_id=="barlett-jessie"]

,member_id,start_date,end_date,member_uri,start_date_precision,end_date_precision,street_address,city,postal_code,latitude,...,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id,lat,lon,arrond_id
dwelling_id,,,,,,,,,,,,,,,,,,,,,
barlett-jessie DWELT AT 26-rue-de-Lübeck FROM 1924-01-01,barlett-jessie,1924-01-01,,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),,26 rue de Lübeck,Paris,75116,48.86522,...,16.0,2289,1048,1359,970,,2.0,48.86522,2.29188,16
barlett-jessie DWELT AT 14-rue-d'Assas FROM 1923-09-01 UNTIL 1923-12-01,barlett-jessie,1923-09-01,1923-12-01,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),DatePrecision(year|month),14 rue d'Assas,Paris,75006,48.84928,...,6.0,2289,1048,3626,852,,2.0,48.84928,2.32841,6


In [50]:
# Number of start dates?
len(df_dwellings[df_dwellings.start_date!=''])

49

In [51]:
# Number of end dates?
len(df_dwellings[df_dwellings.end_date!=''])

46

In [52]:
df_dwellings[df_dwellings.start_date!=''].head(2)

,member_id,start_date,end_date,member_uri,start_date_precision,end_date_precision,street_address,city,postal_code,latitude,...,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id,lat,lon,arrond_id
dwelling_id,,,,,,,,,,,,,,,,,,,,,
barlett-jessie DWELT AT 26-rue-de-Lübeck FROM 1924-01-01,barlett-jessie,1924-01-01,,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),,26 rue de Lübeck,Paris,75116,48.86522,...,16.0,2289,1048,1359,970,,2.0,48.86522,2.29188,16
barlett-jessie DWELT AT 14-rue-d'Assas FROM 1923-09-01 UNTIL 1923-12-01,barlett-jessie,1923-09-01,1923-12-01,https://shakespeareandco.princeton.edu/members...,DatePrecision(year|month),DatePrecision(year|month),14 rue d'Assas,Paris,75006,48.84928,...,6.0,2289,1048,3626,852,,2.0,48.84928,2.32841,6


## New combined df

In [75]:
@cache
def get_borrow_events_df():
    df = get_events_df().reset_index()
    df = df[df.event_type=='Borrow']
    df = df[[c for c in df if c.split('_')[0] not in {'subscription','reimbursement','purchase','currency'} and c not in {'event_type'}]]
    return df.set_index('event_id')
    

In [14]:
def find_dwelling_id(borrow_row, sep='; ', verbose=True):
    memid = borrow_row.member_id
    df = df_dwellings[df_dwellings.member_id==memid]
    
    # if no dwellign at all?
    if not len(df): 
        return ''
    
    # if only one?
    elif len(df)==1: 
        return df.index[0]
    
    ## @TODO: 
    # Prefer Paris, prefer French, ...
        
    
    # ok, if multiple:
    else:
        # if start date?
        borrow_start = borrow_row.start_date
        borrow_end = borrow_row.end_date
        
        if verbose: 
            print('looking for:',borrow_row.event_id,'\n')
            print('possibilities:',sep.join(df.index),'\n')
        
        if borrow_start:
            # If we know when the borrow began, then exclude dwellings which end before that date
            ## CAVEAT: Partial date can make comparison incorrect? @CHECK THIS 
            df = df[df.end_date.apply(lambda x: not (x and x<borrow_start))]
        if borrow_end:
            # If we know when the borrow ended, then exclude dwellings which begin after that date
            ## CAVEAT: Partial date can make comparison incorrect? @CHECK THIS 
            df = df[df.start_date.apply(lambda x: not (x and x>borrow_end))]
        
        o=sep.join(df.index)
        if verbose: print('solution:',o,'\n\n')
        return o


In [65]:
'1922-12-01' < '1922-12-15'

True

In [67]:
'1922-12-01' < '1922-12-15'

False

In [15]:
df_borrows = get_borrow_events_df()
df_borrows

,member_id,book_id,start_date,end_date,borrow_status,borrow_duration_days,source_type,source_citation,source_manifest,source_image,dwelling_ids
event_id,,,,,,,,,,,
lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07,lanux-eyre-de,woolf-night-day,1923,1923-11-07,Returned,,Lending Library Card,"Sylvia Beach, Eyre de Lanux Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c5...,lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...
tery BORROWS james-joyce FROM 1924 TO 1924-10-22,tery,james-joyce,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
tery BORROWS freeman-portrait-george-moore FROM 1924 TO 1924-10-22,tery,freeman-portrait-george-moore,1924,1924-10-22,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/c2...,tery DWELT AT 4-rue-de-l'Hôtel-Colbert
macleish-ada BORROWS stern-tents-israel FROM 1926 TO 1926-01-14,macleish-ada,stern-tents-israel,1926,1926-01-14,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/22...,macleish-ada DWELT AT 10-parc-de-Montretout; m...
macleish-archibald BORROWS stern-tents-israel FROM 1926 TO 1926-01-14,macleish-archibald,stern-tents-israel,1926,1926-01-14,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/22...,macleish-archibald DWELT AT 10-parc-de-Montret...
...,...,...,...,...,...,...,...,...,...,...,...
hommel-bernard BORROWS heine-poems,hommel-bernard,heine-poems,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS cowley-exiles-return-narrative,hommel-bernard,cowley-exiles-return-narrative,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon
hommel-bernard BORROWS wilson-axels-castle-study,hommel-bernard,wilson-axels-castle-study,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e4...,hommel-bernard DWELT AT 12-rue-de-l'Odéon


In [16]:
df_borrows.iloc[0]

member_id                                                   lanux-eyre-de
book_id                                                   woolf-night-day
start_date                                                           1923
end_date                                                       1923-11-07
borrow_status                                                    Returned
borrow_duration_days                                                     
source_type                                          Lending Library Card
source_citation         Sylvia Beach, Eyre de Lanux Lending Library Ca...
source_manifest         https://figgy.princeton.edu/concern/scanned_re...
source_image            https://iiif.princeton.edu/loris/figgy_prod/c5...
dwelling_ids            lanux-eyre-de DWELT AT 126-boulevard-du-Montpa...
Name: lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07, dtype: object

In [17]:
# df_dwellings.query('member_id=="rhys"')

In [18]:
# df_members.loc['rhys']

In [58]:
find_dwelling_id(df_borrows.reset_index().iloc[1])

"tery DWELT AT 4-rue-de-l'Hôtel-Colbert"

In [59]:
find_dwelling_id(df_borrows.reset_index().iloc[2])

"tery DWELT AT 4-rue-de-l'Hôtel-Colbert"

In [60]:
find_dwelling_id(df_borrows.reset_index().iloc[3])

looking for: macleish-ada BORROWS stern-tents-israel FROM 1926 TO 1926-01-14 

possibilities: macleish-ada DWELT AT 10-parc-de-Montretout; macleish-ada DWELT AT 85-boulevard-Saint-Michel; macleish-ada DWELT AT rue-du-Commandant-Yvon; macleish-ada DWELT AT 90-chemin-Gustave-Raymond; macleish-ada DWELT AT 44-rue-de-Bac; macleish-ada DWELT AT 41-avenue-du-Bois-de-Boulogne; macleish-ada DWELT AT 14-rue-Guynemer; macleish-ada DWELT AT 8-rue-Emile-Augier; macleish-ada DWELT AT 22-rue-de-l'Université; macleish-ada DWELT AT ; macleish-ada DWELT AT 135-E-42nd-Street 

solution: macleish-ada DWELT AT 10-parc-de-Montretout; macleish-ada DWELT AT 85-boulevard-Saint-Michel; macleish-ada DWELT AT rue-du-Commandant-Yvon; macleish-ada DWELT AT 90-chemin-Gustave-Raymond; macleish-ada DWELT AT 44-rue-de-Bac; macleish-ada DWELT AT 41-avenue-du-Bois-de-Boulogne; macleish-ada DWELT AT 14-rue-Guynemer; macleish-ada DWELT AT 8-rue-Emile-Augier; macleish-ada DWELT AT 22-rue-de-l'Université; macleish-ada DWELT

"macleish-ada DWELT AT 10-parc-de-Montretout; macleish-ada DWELT AT 85-boulevard-Saint-Michel; macleish-ada DWELT AT rue-du-Commandant-Yvon; macleish-ada DWELT AT 90-chemin-Gustave-Raymond; macleish-ada DWELT AT 44-rue-de-Bac; macleish-ada DWELT AT 41-avenue-du-Bois-de-Boulogne; macleish-ada DWELT AT 14-rue-Guynemer; macleish-ada DWELT AT 8-rue-Emile-Augier; macleish-ada DWELT AT 22-rue-de-l'Université; macleish-ada DWELT AT ; macleish-ada DWELT AT 135-E-42nd-Street"

In [61]:
find_dwelling_id(df_borrows[df_borrows.member_id=='hemingway-ernest'].reset_index().iloc[0])

looking for: hemingway-ernest BORROWS wilder-bridge-san-luis FROM 1927-11 TO 1927-12-07 

possibilities: hemingway-ernest DWELT AT 112-chemin-des-Mougins FROM 1926-08-02 UNTIL 1926-08-12; hemingway-ernest DWELT AT 113-rue-Notre-Dame-des-Champs FROM 1924-02-08 UNTIL 1926-01-01; hemingway-ernest DWELT AT  FROM 1926-05-14 UNTIL 1926-05-28; hemingway-ernest DWELT AT 69-rue-Froidevaux FROM 1926-08-12 UNTIL 1927-01-01; hemingway-ernest DWELT AT  FROM 1931-12-19 UNTIL 1939-12-24; hemingway-ernest DWELT AT Promenade-10 FROM 1926-12-25 UNTIL 1927-03-10; hemingway-ernest DWELT AT Passeig-de-Gràcia-68 FROM 1937-01-01 UNTIL 1937-01-01; hemingway-ernest DWELT AT 44-rue-Jacob FROM 1921-12-22 UNTIL 1922-01-09; hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28 

solution: hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28 




'hemingway-ernest DWELT AT 6-rue-Férou FROM 1927-09-19 UNTIL 1931-05-28'

#### TODO: We should test this more!!

## ... Book Dwellings?

In [73]:
def get_book_dwelling_id(row):
    return f'{row.event_id} WHILE {row.dwelling_id}'

def resolve_ambiguity(options, mode='first'):
    if not options: return options
    if mode=='first': return [options[0]]
    if mode=='last': return [options[-1]]
    if mode=='random': 
        import random
        return [random.choice(options)]
    return options

@cache
def get_book_dwellings_df(sep='; ', ambiguity_mode='random'):
    df_borrows = get_events_df().reset_index()
    df_dwellings = get_dwellings_df().reset_index().drop_duplicates('dwelling_id').set_index('dwelling_id')
    
    odf = pd.DataFrame([
        {**dict(row), **{'dwelling_id':dwelling_id}, **dict(df_dwellings.loc[dwelling_id])}
        for i, row in df_borrows.iterrows()
        for dwelling_id in resolve_ambiguity(row.dwelling_ids.split(sep), mode=ambiguity_mode)
        if row.dwelling_ids and dwelling_id
    ]).fillna('')
    
    odf['book_dwelling_id'] = odf.apply(get_book_dwelling_id, axis=1)
    # odf = odf[[c for c in odf if c.endswith('_id')]]
    return odf.set_index('book_dwelling_id').query('lat!="" & lon!=""')

In [74]:
get_book_dwellings_df()

AttributeError: 'Series' object has no attribute 'dwelling_ids'

In [37]:
df_book_dwellings_all = get_book_dwellings_df(ambiguity_mode='all')
df_book_dwellings_first = get_book_dwellings_df(ambiguity_mode='first')
df_book_dwellings_random = get_book_dwellings_df(ambiguity_mode='random')

In [71]:
df_book_dwellings_first

,event_id,member_id,book_id,start_date,end_date,borrow_status,borrow_duration_days,source_type,source_citation,source_manifest,...,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id,lat,lon,arrond_id
book_dwelling_id,,,,,,,,,,,,,,,,,,,,,
lanux-eyre-de BORROWS woolf-night-day FROM 1923 TO 1923-11-07 WHILE lanux-eyre-de DWELT AT 126-boulevard-du-Montparnasse,lanux-eyre-de BORROWS woolf-night-day FROM 192...,lanux-eyre-de,woolf-night-day,,,Returned,,Lending Library Card,"Sylvia Beach, Eyre de Lanux Lending Library Ca...",https://figgy.princeton.edu/concern/scanned_re...,...,14.0,251,1078,192,295,,2.0,48.84106,2.33146,14
tery BORROWS james-joyce FROM 1924 TO 1924-10-22 WHILE tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery BORROWS james-joyce FROM 1924 TO 1924-10-22,tery,james-joyce,,,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,...,5.0,535,5079,755,695,,2.0,48.85183,2.34894,5
tery BORROWS freeman-portrait-george-moore FROM 1924 TO 1924-10-22 WHILE tery DWELT AT 4-rue-de-l'Hôtel-Colbert,tery BORROWS freeman-portrait-george-moore FRO...,tery,freeman-portrait-george-moore,,,Returned,,Lending Library Card,"Sylvia Beach, Mlle Simone Tery Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,...,5.0,535,5079,755,695,,2.0,48.85183,2.34894,5
macleish-ada BORROWS stern-tents-israel FROM 1926 TO 1926-01-14 WHILE macleish-ada DWELT AT 10-parc-de-Montretout,macleish-ada BORROWS stern-tents-israel FROM 1...,macleish-ada,stern-tents-israel,,,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,...,,3122,1067,379,363,,2.0,48.8408,2.21455,
macleish-archibald BORROWS stern-tents-israel FROM 1926 TO 1926-01-14 WHILE macleish-archibald DWELT AT 10-parc-de-Montretout,macleish-archibald BORROWS stern-tents-israel ...,macleish-archibald,stern-tents-israel,,,Returned,,Lending Library Card,"Sylvia Beach, Ada and Archibald MacLeish Lendi...",https://figgy.princeton.edu/concern/scanned_re...,...,,300,1067,379,363,,2.0,48.8408,2.21455,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hommel-bernard BORROWS heine-poems WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS heine-poems,hommel-bernard,heine-poems,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,...,6.0,230,4886,298,15,,2.0,48.85092,2.33845,6
hommel-bernard BORROWS cowley-exiles-return-narrative WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS cowley-exiles-return-na...,hommel-bernard,cowley-exiles-return-narrative,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,...,6.0,230,4886,298,15,,2.0,48.85092,2.33845,6
hommel-bernard BORROWS wilson-axels-castle-study WHILE hommel-bernard DWELT AT 12-rue-de-l'Odéon,hommel-bernard BORROWS wilson-axels-castle-study,hommel-bernard,wilson-axels-castle-study,,,Returned,,Lending Library Card,"Sylvia Beach, Bernard Hommel Lending Library C...",https://figgy.princeton.edu/concern/scanned_re...,...,6.0,230,4886,298,15,,2.0,48.85092,2.33845,6


In [38]:
len(df_book_dwellings_all), len(df_book_dwellings_first), len(df_book_dwellings_random)

(54441, 21020, 20961)

In [39]:
df=df_book_dwellings_all
# df[df.book_id=='joyce-ulysses']

In [40]:
df=df_book_dwellings_first
len(df[df.book_id=='joyce-ulysses'])

25

In [41]:
len(get_borrow_events_df().query('book_id=="joyce-ulysses"'))

25

In [78]:
compare_choropleths(
    df_book_dwellings_all.query('book_id=="joyce-ulysses"'),
    df_book_dwellings_all.query('book_id=="woolf-mrs-dalloway"'),
    desc1='Ulysses', desc2='Dalloway',
)

In [70]:
df_book_dwellings_all[df_book_dwellings_all.country_id==16]

,event_id,member_id,book_id,start_date,end_date,borrow_status,borrow_duration_days,source_type,source_citation,source_manifest,...,arrrondissement,person_id,account_id,address_id,location_id,care_of_person_id,country_id,lat,lon,arrond_id
book_dwelling_id,,,,,,,,,,,,,,,,,,,,,
killen BORROWS lee-life-william-shakespeare FROM 1922-11 TO 1922-11-16 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS lee-life-william-shakespeare FR...,killen,lee-life-william-shakespeare,,,Returned,,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS lewis-babbitt FROM 1939-12 TO 1939-12-11 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS lewis-babbitt FROM 1939-12 TO 1...,killen,lewis-babbitt,,,Returned,,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS walpole-prayer FROM 1940-02 TO 1940-03-01 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS walpole-prayer FROM 1940-02 TO ...,killen,walpole-prayer,,,Returned,,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS oriordan-london-story-adam FROM 1922-11-04 TO 1922-11-06 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS oriordan-london-story-adam FROM...,killen,oriordan-london-story-adam,,,Returned,2.0,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS moore-muslin ON 1922-11-06 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS moore-muslin ON 1922-11-06,killen,moore-muslin,,,Returned,,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
killen BORROWS christie-sad-cypress FROM 1940-03-20 TO 1940-03-21 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS christie-sad-cypress FROM 1940-...,killen,christie-sad-cypress,,,Returned,1.0,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS eberhart-chiffon-scarf FROM 1940-03-21 TO 1940-03-26 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS eberhart-chiffon-scarf FROM 194...,killen,eberhart-chiffon-scarf,,,Returned,5.0,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
killen BORROWS walpole-head-green-bronze FROM 1940-03-21 TO 1940-03-26 WHILE killen DWELT AT 60-Cherryvalley-Park,killen BORROWS walpole-head-green-bronze FROM ...,killen,walpole-head-green-bronze,,,Returned,5.0,Lending Library Card,"Sylvia Beach, Alice M. Killen Lending Library ...",https://figgy.princeton.edu/concern/scanned_re...,...,,245,814,321,288,,16.0,54.5855,-5.84983,
